In [1]:
import pandas as pd
import numpy as np

In [43]:
# # https://dacon.io/competitions/official/235720/codeshare/2583?page=1&dtype=recent 참고

# load
data1 = pd.read_csv('한국동서발전 시간대별 태양광 및 풍력 발전량 현황(2018_2019).csv', encoding='euc-kr')
data2 = pd.read_csv('한국동서발전 시간대별 태양광 발전량 현황(2015_2017).csv', encoding='euc-kr')

# convert name
convert_name = {'당진태양광':'dangjin', '당진수상태양광':'dangjin_floating', '당진자재창고태양광':'dangjin_warehouse', '울산태양광':'ulsan'}

In [44]:
def preprocess(energy_type):
    # sample
    data1_ = data1.loc[data1['태양광명'] == energy_type]
    data2_ = data2.loc[data2['태양광명'] == energy_type]

    # merge two data
    data1_.drop(columns=['태양광명'], inplace=True)
    data2_.drop(columns=['태양광명','용량(MW)'], inplace=True)
    data = data2_.append(data1_)

    # datetime
    data['시간'] = pd.to_datetime(data['시간'])

    # modify df
    hours = [] # to store 'time'
    values = [] # to store energy value

    for i in range(data.shape[0]):
        row = data.iloc[i]
        day = row['시간']

        for h in range(24): # loop over one row(one day)
            hour = day.replace(hour=h)
            hour += pd.Timedelta(1, unit='hour') # data is originaly 1~24. Change to 0~23.

            hours.append(hour)
            values.append(row[str(h+1)])

    df = pd.DataFrame({'time':hours, convert_name[energy_type]:values})
    return df.set_index('time')

In [45]:
dangjin = preprocess('당진태양광')
dangjin_floating = preprocess('당진수상태양광')
dangjin_warehouse = preprocess('당진자재창고태양광')
ulsan = preprocess('울산태양광')

/home/yejoon/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [46]:
# shape
print(dangjin.shape)
print(dangjin_floating.shape)
print(dangjin_warehouse.shape)
print(ulsan.shape)

(40824, 1)
(14592, 1)
(40776, 1)
(39408, 1)


In [47]:
# merge
dangjin_all = pd.concat([dangjin,dangjin_floating,dangjin_warehouse], axis=1, join='outer')

# slice time 
dangjin_all = dangjin_all.loc[dangjin_all.index < pd.Timestamp('2018-03-02 00:00:00')]
ulsan = ulsan.loc[ulsan.index < pd.Timestamp('2018-03-02 00:00:00')]

## Fill na in dangjin_floating

In [48]:
print(dangjin_all.isna().sum())
print(ulsan.isna().sum())

dangjin                  0
dangjin_floating     26232
dangjin_warehouse        0
dtype: int64
ulsan    0
dtype: int64


In [49]:
dangjin_all_mean_series = dangjin_all.mean(axis=0)

djdjw_to_djflt = dangjin_all_mean_series['dangjin_floating']  / (dangjin_all_mean_series['dangjin'] + dangjin_all_mean_series['dangjin_warehouse'])

djdjw_to_djflt

0.48344334791995036

In [50]:
na_dangjin_floating = dangjin_all.loc[dangjin_all['dangjin_floating'].isna()] # df where dangjin_floating = na
na_dangjin_floating.loc[:,'dangjin_floating'] = (na_dangjin_floating['dangjin'] + na_dangjin_floating['dangjin_warehouse']) * djdjw_to_djflt

dangjin_all_filled = dangjin_all.copy()
dangjin_all_filled[dangjin_all_filled['dangjin_floating'].isna()] = na_dangjin_floating

/home/yejoon/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [60]:
# pickle
import pickle

with open('energy_dj_extenral.pkl', 'wb') as f:
    pickle.dump(dangjin_all_filled, f)

with open('energy_uls_external.pkl', 'wb') as f:
    pickle.dump(ulsan, f)